## Recommender System - Marketplace Matching

In this notebook, we will: 
- Clean textual data from user-input verbatim posts
- Use a Word2Vec model to calculate document similarities 
- Sort the most similar user input to our training data in order to recommend similar products 
- Save this model in a format that allows us to refresh the testing data

The goal of this project is to create a recommender system to help Pangeans find the right "project" for them, given their profile information. Here, we are using legacy data that is from Pangea V2, when Pangeans were allowed to post services and requests, as well as purchase items on the platform. We are using the User-Inputted Titles to suggest similar services, or in V3, similar "projects". 

In [1]:
'''Importing Libraries'''
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords') ###
from nltk.tokenize import RegexpTokenizer

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

import os, sys

from operator import add

from sklearn.preprocessing import normalize

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from adjustText import adjust_text

from sklearn.manifold import TSNE

import pickle

import json
from pprint import pprint
from pandas.io.json import json_normalize

import operator

import csv

import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelateng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
'''Loading the model'''
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [3]:
'''Vectorize and store existing titles in legacy Pangea database'''
def vectorize_and_store_existing_titles(): 
    raw = pd.read_csv("allPostData.csv", header=0);
    titles = raw['title'];
    post_titles = [title for title in titles];
    post_titles = set(post_titles); 
    tokens = [[word for word in title.lower().split()] for title in post_titles];
    clean_words = [[word.translate(str.maketrans('', '', string.punctuation)) for word in title] for title in tokens]
    #print(clean_words)
    #clean_words = [[word for word in title if word.isalpha()] for title in tokens];
    #tokenizer = RegexpTokenizer(r'\w+')
    
    stoplist = set(stopwords.words('english'));
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];   
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    dictionary = dict(zip(post_titles, filtered_word_list))
    vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    for title in post_titles: 
        word_vecs = [model[word] for word in dictionary[title]]
        if len(word_vecs) == 0:
            title_vec = [np.zeros(300)]
        else: 
            title_vec = normalize(sum(word_vecs).reshape(1, -1))
        vectorized_titles = vectorized_titles.append({'Titles': title, 'Vectors': title_vec}, ignore_index=True)
    vectorized_titles.to_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
    return(vectorized_titles)


In [4]:
'''Print a dataframe to show how these titles and their vectors will be saved'''
'''Also note that we are saving the df with the original raw (not cleaned) titles'''
vectorize_and_store_existing_titles()

,Titles,Vectors
0,Fashion Stylist,"[[-0.0208001, 0.027787589, -0.06782213, 0.0072..."
1,Photography Sessions,"[[-0.0072251307, 0.033729766, -0.047951847, 0...."
2,Beginner Chess Lessons 1,"[[0.014434023, -0.051012553, 0.08174653, 0.028..."
3,SAS Advice!,"[[-0.059581243, -0.05003842, -0.029495988, 0.0..."
4,CVS Picker Upper,"[[0.01778612, -0.03279248, -0.06992833, 0.1205..."
5,Chef Prepared Meal Prep,"[[-0.10330923, 0.029792165, 0.012260806, 0.072..."
6,App Development Consultant,"[[0.02185485, -0.07096081, -0.08069656, -0.026..."
7,INSTAGRAM,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8,Eliminate Russell Carey & Replace Him With a R...,"[[-0.04523085, 0.047022626, 0.030563282, 0.070..."
9,Math tutor,"[[0.05884398, -0.012645489, 0.022123868, 0.072..."


In [5]:
'''Read a JSON file when a new user pings with a request'''
with open('firstPost.json') as fresh_data:
    user_post = json.load(fresh_data)

'''Vectorize each new title as a user/student/company creates a new post'''
def vectorize_new_title(user_post):
    #ranked_titles_load = pd.read_csv("./ranked_titles.csv")
    json_df = pd.DataFrame.from_dict(json_normalize(user_post), orient='columns')
    title = json_df["title"][0] #--> added [0] because json_df["title"] was still in pd.Series form
    json_tokens = [word for word in title.lower().split()]
    #json_clean_words = [[word for word in title.lower().split()] for title in post_titles];
    json_clean_words = [word.translate(str.maketrans('', '', string.punctuation)) for word in json_tokens]
    #print(json_clean_words)
    
    stoplist = set(stopwords.words('english'));
    json_titles_nostopwords = [word for word in json_clean_words if word not in stoplist]   
    json_preprocessed = [word for word in json_titles_nostopwords if word in model.vocab]   
    json_title_vectors = {}
    json_vectorized_title_df = pd.DataFrame(columns=["Titles", "Vectors"])
    json_word_vecs = [model[word] for word in json_preprocessed]
    if len(json_preprocessed) == 0:
            json_title_vec = [np.zeros(300)]
    else: 
        json_title_vec = normalize(sum(json_word_vecs).reshape(1, -1))
    json_vectorized_title_df = json_vectorized_title_df.append({'Titles': title, 'Vectors': json_title_vec}, ignore_index = True)
    #export_csv = json_vectorized_title_df.to_csv (r'/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
    if not os.path.isfile('/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv'):
        json_vectorized_title_df.to_csv (r'/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv', index = None, header=True)
    else:
        json_vectorized_title_df.to_csv (r'/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv', mode='a', index = None, header=False)
    
    return(json_vectorized_title_df)

In [6]:
'''Append new user-post title to csv'''
vectorized_title = vectorize_new_title(user_post)
print(vectorized_title)

                  Titles                                            Vectors
0  Teach Me How To Cook!  [[-0.022974137, 0.010501054, 0.05975248, 0.041...


In [7]:
'''Load the current titles in the Pangea database, and then rank them by similarity to the latest user query'''
def rank_existing_titles(vectorized_title):
    ranked_titles = {}
    other_titles = pd.read_pickle("./vectorized_titles.pkl")
    for index,row in other_titles.iterrows():
        ranked_titles[row['Titles']] = sum(row['Vectors'][0]*vectorized_title['Vectors'][0][0]) # --> did the dot product using sum() and * because np.dot was behaving weirdly for some reason. Now it seems to work! 
    sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    return(sorted_title_vecs)

#should this also be named ranked titles? or should we rename this line? since we're 
#already loading them in the cell above ??


In [8]:
#rank_existing_titles(vectorized_title)

In [9]:
'''Final function call API that puts together the prior 3 functions in a neat mega-function'''
def generate_recommendations(user_post):
    title = user_post["title"]
    vectorized_title = vectorize_new_title(user_post)
    ranked_titles = rank_existing_titles(vectorized_title)    
    with open("./ranked_titles.csv", "w", newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for title in ranked_titles:
            wr.writerow([ranked_titles, title])
    return(ranked_titles)

#should this still be ranked titles.csv???


In [10]:
#sanity check for final mega function
generate_recommendations(user_post)

[('Teach Me How To Cook!', 1.0000000256776254),
 ('Teach me how to cook quick meals!', 0.8501101096892398),
 ('Teach how to prepare steaks ', 0.7433686636395578),
 ('Private cook', 0.7177446654708319),
 ('Private Cook ', 0.7177446654708319),
 ('Cooking Lessons ', 0.6707953416986925),
 ('Cooking Lessons', 0.6707953416986925),
 ('Cooking lessons', 0.6707953416986925),
 ('Cooking', 0.6634962084033305),
 ('Teach me how to play piano', 0.6475645409999515),
 ('Teach how to make tea', 0.6299678694610975),
 ('Culinary tutor', 0.5939296423903215),
 ('Baking Lessons ', 0.5822712795805955),
 ('Mediterranean Cooking Class', 0.5702011714132595),
 ('Teach you how to shuffle', 0.5699134945516562),
 ('Teach Me How To Solve Rubiks Cube', 0.55600204987811),
 ('Brazilian Cooking Class', 0.5491360732714838),
 ('Learn To Juggle!', 0.5355930420098503),
 ('Chef Prepared Meal Prep', 0.5327085255917154),
 ('Help me learn Pop/Stars dance', 0.5109324808180418),
 ('Teach me meme culture', 0.5040128903044376),
 ('

In [11]:
#sanity check
df = pd.DataFrame(generate_recommendations(user_post))
df.columns = ["Title", "Similarity Score"]
print(df)

                                 Title  Similarity Score
0                Teach Me How To Cook!          1.000000
1    Teach me how to cook quick meals!          0.850110
2         Teach how to prepare steaks           0.743369
3                         Private cook          0.717745
4                        Private Cook           0.717745
5                     Cooking Lessons           0.670795
6                      Cooking Lessons          0.670795
7                      Cooking lessons          0.670795
8                              Cooking          0.663496
9           Teach me how to play piano          0.647565
10               Teach how to make tea          0.629968
11                      Culinary tutor          0.593930
12                     Baking Lessons           0.582271
13         Mediterranean Cooking Class          0.570201
14            Teach you how to shuffle          0.569913
15   Teach Me How To Solve Rubiks Cube          0.556002
16             Brazilian Cookin